<a href="https://colab.research.google.com/github/ThaDuyx/Classify/blob/dev/Cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross validation script


In [ ]:
%%capture
# install libraries
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install accelerate -U

In [ ]:
# import libraries
import numpy as np
import evaluate
from datasets import load_dataset
from sklearn.model_selection import KFold
from transformers import Trainer, TrainingArguments, AutoModelForAudioClassification, AutoFeatureExtractor, AutoConfig
from huggingface_hub import notebook_login
import torch

In [ ]:
# authenticate
notebook_login()

In [ ]:
# setup distil-HuBERT model
%%capture
model_id = "ntu-spml/distilhubert"
model_name = model_id.split("/")[-1]
datasetName = "TheDuyx/augmented_bass_sounds"
datasetTag = "augmented_bass_sounds"
pre_name="cross-bass10"

In [ ]:
# load dataset
%%capture
dataset = load_dataset(datasetName)
data = dataset['train']
labels = np.array(data['label'])
kf = KFold(n_splits=7, shuffle=True, random_state=42)

In [ ]:
#def compute_metrics(pred):
  #labels = pred.label_ids
  #preds = pred.predictions.argmax(-1)
  #accuracy = (preds == labels).astype(np.float32).mean().item()
  #return {'accuracy': accuracy}


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
results = []

feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
id2label_fn = dataset["train"].features["label"].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(dataset["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

for fold, (train_index, val_index) in enumerate(kf.split(labels)):
  train_dataset = data.select(train_index.tolist())
  val_dataset = data.select(val_index.tolist())

  model = AutoModelForAudioClassification.from_pretrained(
      model_id,
      num_labels=num_labels,
      label2id=label2id,
      id2label=id2label)

  training_args = TrainingArguments(
    output_dir=f'./results_fold3_{fold}',         # output directory
    num_train_epochs=4,                           # total number of training epochs
    per_device_train_batch_size=8,                # batch size for training
    per_device_eval_batch_size=16,                # batch size for evaluation
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=0.02,                            # strength of weight decay
    learning_rate=5e-5,
    gradient_accumulation_steps=4,
    logging_dir=f'./logs_fold_{fold}',            # directory for storing logs
    evaluation_strategy="epoch",                  # evaluate every epoch
    save_strategy="epoch",                        # save checkpoints every epoch
    fp16=True,
    push_to_hub=True,
  )

  metric = evaluate.load("accuracy")

  # Initialize Trainer
  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
  )

  trainer.train()
  eval_result = trainer.evaluate()
  results.append(eval_result)


for fold, result in enumerate(results):
  print(f"Fold {fold}: {result}")

average_accuracy = np.mean([result['eval_accuracy'] for result in results])
print(f"Average accuracy: {average_accuracy}")

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.890700,0.021406,0.994970
1,0.012000,0.010181,0.998171
2,0.002300,0.002090,0.999543
3,0.000200,0.000070,1.000000


...

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.994600,0.044686,0.989483
1,0.007000,0.009002,0.998400
2,0.001600,0.004103,0.999314
3,0.000700,0.002131,0.999771


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,1.029900,0.083326,0.979652
1,0.006800,0.001033,0.999771
2,0.002400,0.004468,0.998857
3,0.000200,0.001488,0.999543


Fold 0: {'eval_loss': 7.006908708717674e-05, 'eval_accuracy': 1.0, 'eval_runtime': 62.484, 'eval_samples_per_second': 70.002, 'eval_steps_per_second': 4.385, 'epoch': 3.998780859494057}
Fold 1: {'eval_loss': 0.0028043121565133333, 'eval_accuracy': 0.9995427526291724, 'eval_runtime': 61.5453, 'eval_samples_per_second': 71.07, 'eval_steps_per_second': 4.452, 'epoch': 3.998780859494057}
Fold 2: {'eval_loss': 0.0013350064400583506, 'eval_accuracy': 0.9997713763145862, 'eval_runtime': 64.186, 'eval_samples_per_second': 68.146, 'eval_steps_per_second': 4.269, 'epoch': 3.998780859494057}
Fold 3: {'eval_loss': 0.005046503618359566, 'eval_accuracy': 0.9993141289437586, 'eval_runtime': 64.0579, 'eval_samples_per_second': 68.282, 'eval_steps_per_second': 4.277, 'epoch': 3.998780859494057}
Fold 4: {'eval_loss': 0.0020590159110724926, 'eval_accuracy': 0.9993141289437586, 'eval_runtime': 63.799, 'eval_samples_per_second': 68.559, 'eval_steps_per_second': 4.295, 'epoch': 3.998780859494057}
Fold 5: {'